# Data Loading
We use `pandas` package: https://pandas.pydata.org/docs/index.html

Firstly, we can use `pandas.Excelfile()` to load the Excel file

In [3]:
import pandas

exceldata = pandas.ExcelFile('caschool.xlsx')

Load the Excel data into dataframe for further operations, then show the first 5 rows
Here `df` is a `pandas.Dataframe` object:

In [4]:
df = exceldata.parse('caschool')
# Uncomment the line below to see type of df
# print(type(df))

# Show the first five rows of the parsed data
df.head(5)

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Observation Number,dist_cod,county,district,gr_span,enrl_tot,teachers,calw_pct,meal_pct,computer,testscr,comp_stu,expn_stu,str,avginc,el_pct,read_scr,math_scr
0,1,75119,Alameda,Sunol Glen Unified,KK-08,195,10.900000,0.510200,2.040800,67,690.799988,0.343590,6384.911133,17.889910,22.690001,0.000000,691.599976,690.000000
1,2,61499,Butte,Manzanita Elementary,KK-08,240,11.150000,15.416700,47.916698,101,661.200012,0.420833,5099.380859,21.524664,9.824000,4.583333,660.500000,661.900024
2,3,61549,Butte,Thermalito Union Elementary,KK-08,1550,82.900002,55.032299,76.322601,169,643.599976,0.109032,5501.954590,18.697226,8.978000,30.000002,636.299988,650.900024
3,4,61457,Butte,Golden Feather Union Elementary,KK-08,243,14.000000,36.475399,77.049202,85,647.700012,0.349794,7101.831055,17.357143,8.978000,0.000000,651.900024,643.500000
4,5,61523,Butte,Palermo Union Elementary,KK-08,1335,71.500000,33.108601,78.427002,171,640.849976,0.128090,5235.987793,18.671329,9.080333,13.857677,641.799988,639.900024


# Summary Statistics
Summary statistics for some columns. Transpose them and do some conversion to make them more similar to the R version.

In [5]:
df_summary = df[['testscr','str','el_pct','meal_pct','calw_pct']].describe()
df_summary = df_summary.transpose()

# Compute skewness and kurtosis
skew = pandas.DataFrame(df[['testscr','str','el_pct','meal_pct','calw_pct']].skew(skipna = True), columns=['skew'])
kurtosis = pandas.DataFrame(df[['testscr','str','el_pct','meal_pct','calw_pct']].kurtosis(skipna = True), columns=['kurtosis'])

# Add skew and kurtosis as new columns in summary dataframe
df_summary = pandas.concat([df_summary,skew],axis=1)
df_summary = pandas.concat([df_summary,kurtosis],axis=1)

# rounding of count column to integer
df_summary = df_summary.astype({'count':'int32'})
# Show the table
print(df_summary)

# Save the summary table as CSV
df_summary.to_csv('pythonver_summary.csv')


          count        mean        std         min         25%         50%  \
testscr     420  654.156548  19.053348  605.550049  640.049988  654.449982   
str         420   19.640425   1.891812   14.000000   18.582360   19.723208   
el_pct      420   15.768155  18.285927    0.000000    1.940807    8.777634   
meal_pct    420   44.705237  27.123381    0.000000   23.282200   41.750700   
calw_pct    420   13.246042  11.454821    0.000000    4.395375   10.520450   

                 75%         max      skew  kurtosis  
testscr   666.662506  706.750000  0.091944 -0.242919  
str        20.871815   25.799999 -0.025457  0.631340  
el_pct     22.970003   85.539719  1.431917  1.467060  
meal_pct   66.864725  100.000000  0.184614 -0.997384  
calw_pct   18.981350   78.994202  1.689099  4.659125  


# Correlation Matrix

In [6]:
corr_matrix = df[['testscr','str','el_pct','meal_pct','calw_pct']].corr(method='pearson')
# Show the matrix
print(corr_matrix)
# Save the matrix to csv
corr_matrix.to_csv('pythonver_corr_matrix.csv')

           testscr       str    el_pct  meal_pct  calw_pct
testscr   1.000000 -0.226363 -0.644124 -0.868772 -0.626853
str      -0.226363  1.000000  0.187642  0.135203  0.018276
el_pct   -0.644124  0.187642  1.000000  0.653061  0.319576
meal_pct -0.868772  0.135203  0.653061  1.000000  0.739422
calw_pct -0.626853  0.018276  0.319576  0.739422  1.000000


# Regression Models
We're using `statsmodels` package

In [7]:
import statsmodels.formula.api as smf

## Regression 1
### Model 1_1
Student-teacher ratio as regressor. Use `HC1` heteroscedascity robust covariance.

In [8]:
# Fit regression model
model1_1 = smf.ols("testscr ~ str", data=df).fit(cov_type='HC1')

# Inspect the results
print(model1_1.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     19.26
Date:                Fri, 17 Sep 2021   Prob (F-statistic):           1.45e-05
Time:                        21:24:26   Log-Likelihood:                -1822.2
No. Observations:                 420   AIC:                             3648.
Df Residuals:                     418   BIC:                             3657.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    698.9330     10.364     67.436      0.0

Printing just the coefficients

In [9]:
print(model1_1.params)

Intercept    698.932952
str           -2.279808
dtype: float64


Printing the standard error of parameter

In [10]:
print(model1_1.bse)

Intercept    10.364360
str           0.519489
dtype: float64


RMSE of residual can be computed from mean square error of residual

In [12]:
import math
print(model1_1.mse_resid)
print(math.sqrt(model1_1.mse_resid))

345.25235311665574
18.58096749678702


**Note:** If needed, you can access underlying tables of `summary()` using `summary().tables` attribute. It is a list of `statsmodels.iolib.table.SimpleTable` objects, which has methods to, for example, render their content as HTML. Example:

In [9]:
summary_tables = model1_1.summary().tables
print(summary_tables)
print(summary_tables[0].as_html())

[<class 'statsmodels.iolib.table.SimpleTable'>, <class 'statsmodels.iolib.table.SimpleTable'>, <class 'statsmodels.iolib.table.SimpleTable'>]
<table class="simpletable">
<caption>OLS Regression Results</caption>
<tr>
  <th>Dep. Variable:</th>         <td>testscr</td>     <th>  R-squared:         </th> <td>   0.051</td>
</tr>
<tr>
  <th>Model:</th>                   <td>OLS</td>       <th>  Adj. R-squared:    </th> <td>   0.049</td>
</tr>
<tr>
  <th>Method:</th>             <td>Least Squares</td>  <th>  F-statistic:       </th> <td>   19.26</td>
</tr>
<tr>
  <th>Date:</th>             <td>Fri, 17 Sep 2021</td> <th>  Prob (F-statistic):</th> <td>1.45e-05</td>
</tr>
<tr>
  <th>Time:</th>                 <td>19:53:38</td>     <th>  Log-Likelihood:    </th> <td> -1822.2</td>
</tr>
<tr>
  <th>No. Observations:</th>      <td>   420</td>      <th>  AIC:               </th> <td>   3648.</td>
</tr>
<tr>
  <th>Df Residuals:</th>          <td>   418</td>      <th>  BIC:               </th> <td>   

### Model 1_2
Student-teacher ratio and percentage of English learners as regressors

In [29]:
model1_2 = smf.ols("testscr ~ str + el_pct", data=df).fit(cov_type='HC1')
print(model1_2.summary())
print("\nPrinting just the coefficients")
print(model1_2.params)
print("\nPrinting the standard errors")
print(model1_2.bse)

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.426
Model:                            OLS   Adj. R-squared:                  0.424
Method:                 Least Squares   F-statistic:                     223.8
Date:                Fri, 17 Sep 2021   Prob (F-statistic):           9.28e-67
Time:                        19:54:55   Log-Likelihood:                -1716.6
No. Observations:                 420   AIC:                             3439.
Df Residuals:                     417   BIC:                             3451.
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    686.0322      8.728     78.599      0.0

### Model 1_3

In [11]:
model1_3 = smf.ols("testscr ~ str+el_pct+meal_pct", data=df).fit(cov_type='HC1')
print(model1_3.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     453.5
Date:                Fri, 17 Sep 2021   Prob (F-statistic):          1.05e-130
Time:                        19:53:38   Log-Likelihood:                -1520.5
No. Observations:                 420   AIC:                             3049.
Df Residuals:                     416   BIC:                             3065.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    700.1500      5.568    125.735      0.0

### Model 1_4

In [12]:
model1_4 = smf.ols("testscr ~ str+el_pct+calw_pct", data=df).fit(cov_type='HC1')
print(model1_4.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.626
Method:                 Least Squares   F-statistic:                     170.4
Date:                Fri, 17 Sep 2021   Prob (F-statistic):           4.93e-72
Time:                        19:53:38   Log-Likelihood:                -1625.3
No. Observations:                 420   AIC:                             3259.
Df Residuals:                     416   BIC:                             3275.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    697.9987      6.920    100.861      0.0

### Model 1_5

In [13]:
model1_5 = smf.ols("testscr ~ str+el_pct+meal_pct+calw_pct", data=df).fit(cov_type='HC1')
print(model1_5.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     361.7
Date:                Fri, 17 Sep 2021   Prob (F-statistic):          8.86e-134
Time:                        19:53:38   Log-Likelihood:                -1520.2
No. Observations:                 420   AIC:                             3050.
Df Residuals:                     415   BIC:                             3071.
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    700.3918      5.537    126.483      0.0

### Generating the combined table and writing the output to HTML
Here we use the `stargazer` package from `https://pypi.org/project/stargazer/`, which is the Python implementation of R's `stargazer`.

This creates the equivalent of Table 7.1 in the lecture slides.

In [14]:
from stargazer.stargazer import Stargazer
stargazer = Stargazer([model1_1,model1_2,model1_3,model1_4,model1_5])
stargazer.significant_digits(2)
# Adjust the covariate order. The names can be seen in `stargazer2.cov_names`
stargazer.covariate_order(['str','el_pct','meal_pct','calw_pct','Intercept'])

# Uncomment to see the resulting HTML
# print(stargazer.render_html())

with open('pythonver_reg_1.html','w') as f_out:
    f_out.write(stargazer.render_html())

## Regression 2
### Adding new features to the dataframe

In [15]:
import math

df['avginclog'] = [math.log(x) for x in df['avginc']]
df['HiEL'] = (df['el_pct'] > 10).astype(int)
df['str2'] = [x**2 for x in df['str']]
df['str3'] = [x**3 for x in df['str']]

print(df.head(5))

   Observation Number  dist_cod   county                         district  \
0                   1     75119  Alameda               Sunol Glen Unified   
1                   2     61499    Butte             Manzanita Elementary   
2                   3     61549    Butte      Thermalito Union Elementary   
3                   4     61457    Butte  Golden Feather Union Elementary   
4                   5     61523    Butte         Palermo Union Elementary   

  gr_span  enrl_tot   teachers   calw_pct   meal_pct  computer  ...  \
0   KK-08       195  10.900000   0.510200   2.040800        67  ...   
1   KK-08       240  11.150000  15.416700  47.916698       101  ...   
2   KK-08      1550  82.900002  55.032299  76.322601       169  ...   
3   KK-08       243  14.000000  36.475399  77.049202        85  ...   
4   KK-08      1335  71.500000  33.108601  78.427002       171  ...   

      expn_stu        str     avginc     el_pct    read_scr    math_scr  \
0  6384.911133  17.889910  22.69000

### Model 2_1

In [16]:
model2_1 = smf.ols("testscr ~ str + el_pct + meal_pct", data=df).fit(cov_type='HC1')
print(model2_1.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     453.5
Date:                Fri, 17 Sep 2021   Prob (F-statistic):          1.05e-130
Time:                        19:53:38   Log-Likelihood:                -1520.5
No. Observations:                 420   AIC:                             3049.
Df Residuals:                     416   BIC:                             3065.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    700.1500      5.568    125.735      0.0

### Model 2_2
Using the additonal feature `avignclog`

In [17]:
model2_2 = smf.ols("testscr ~ str + el_pct + meal_pct + avginclog", data=df).fit(cov_type='HC1')
print(model2_2.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.796
Model:                            OLS   Adj. R-squared:                  0.794
Method:                 Least Squares   F-statistic:                     417.2
Date:                Fri, 17 Sep 2021   Prob (F-statistic):          6.40e-144
Time:                        19:53:38   Log-Likelihood:                -1499.3
No. Observations:                 420   AIC:                             3009.
Df Residuals:                     415   BIC:                             3029.
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    658.5520      8.642     76.208      0.0

### Model 2_3

In [18]:
model2_3 = smf.ols("testscr ~ str*HiEL", data=df).fit(cov_type='HC1')
print(model2_3.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.305
Method:                 Least Squares   F-statistic:                     63.67
Date:                Fri, 17 Sep 2021   Prob (F-statistic):           6.74e-34
Time:                        19:53:39   Log-Likelihood:                -1755.3
No. Observations:                 420   AIC:                             3519.
Df Residuals:                     416   BIC:                             3535.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    682.2458     11.868     57.487      0.0

### Model 2_4

In [19]:
model2_4 = smf.ols("testscr ~ str*HiEL + meal_pct + avginclog", data=df).fit(cov_type='HC1')
print(model2_4.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.797
Model:                            OLS   Adj. R-squared:                  0.795
Method:                 Least Squares   F-statistic:                     335.8
Date:                Fri, 17 Sep 2021   Prob (F-statistic):          3.32e-143
Time:                        19:53:39   Log-Likelihood:                -1498.1
No. Observations:                 420   AIC:                             3008.
Df Residuals:                     414   BIC:                             3032.
Df Model:                           5                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    653.6661      9.869     66.232      0.0

### Model 2_5

In [20]:
model2_5 = smf.ols("testscr ~ str + str2 + str3 + HiEL + meal_pct + avginclog", data=df).fit(cov_type='HC1')
print(model2_5.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     281.1
Date:                Fri, 17 Sep 2021   Prob (F-statistic):          2.03e-142
Time:                        19:53:39   Log-Likelihood:                -1494.2
No. Observations:                 420   AIC:                             3002.
Df Residuals:                     413   BIC:                             3031.
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    252.0509    163.634      1.540      0.1

### Model 2_6

In [21]:
model2_6 = smf.ols("testscr ~ str*HiEL + str2*HiEL + str3*HiEL + meal_pct + avginclog", data=df).fit(cov_type='HC1')
print(model2_6.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                     199.8
Date:                Fri, 17 Sep 2021   Prob (F-statistic):          6.56e-144
Time:                        19:53:39   Log-Likelihood:                -1492.1
No. Observations:                 420   AIC:                             3004.
Df Residuals:                     410   BIC:                             3045.
Df Model:                           9                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    122.3542    185.519      0.660      0.5

### Model 2_7

In [22]:
model2_7 = smf.ols("testscr ~ str + str2 + str3 + el_pct + meal_pct + avginclog", data=df).fit(cov_type='HC1')
print(model2_7.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     280.8
Date:                Fri, 17 Sep 2021   Prob (F-statistic):          2.47e-142
Time:                        19:53:39   Log-Likelihood:                -1494.6
No. Observations:                 420   AIC:                             3003.
Df Residuals:                     413   BIC:                             3031.
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    244.8090    165.722      1.477      0.1

### Generating the combined table and writing the output to HTML
As with the previous section, we use `stargazer`
This produces the equivalent of Table 8.3 in the lecture slides

In [23]:
stargazer2 = Stargazer([model2_1,model2_2,model2_3,model2_4,model2_5,model2_6,model2_7])
stargazer2.significant_digits(2)
# Adjust the covariate order. The names can be seen in `stargazer2.cov_names`
stargazer2.covariate_order(['str','el_pct','str2','str3','meal_pct','avginclog',
                           'HiEL','str:HiEL','str2:HiEL','str3:HiEL','Intercept'])

with open('pythonver_reg_2.html','w') as f_out:
    f_out.write(stargazer2.render_html())

## Hypothesis testing
### Example using `model2_6`
Test for all `str` variables  and interactions = 0

In [24]:
hypotheses = '(str:HiEL = 0), (str2:HiEL = 0), (str3:HiEL = 0), (str=0),(str2=0),(str3=0)'
f_test1 = model2_6.f_test(hypotheses)
print(f_test1)

<F test: F=array([[4.96166765]]), p=6.415828313393217e-05, df_denom=410, df_num=6>


Test STR2=0, STR3=0

In [25]:
hypotheses = '(str2=0),(str3=0)'
f_test2 = model2_6.f_test(hypotheses)
print(f_test2)

<F test: F=array([[5.80629655]]), p=0.003261428780200182, df_denom=410, df_num=2>
